# Generalisability Theory Demo

## 1. Create a Data Frame (Simulated Data)

The dataset is based on a two-facet design (see “Two-facet p x i x o
design” in Huebner & Lucht, 2019 for more details), with raters and
criteria items as facets, and theories/gaps as the object of
measurement.

The initial scenario assumes that each of 3 raters evaluates 10 theories
over 4 criteria items (on a scale of 1 to 10), resulting in a 3 x 4 x 10
= 120-row dataset. The scores are taken from Table 3.2 of Brennan
(2001).

### Running R Code

In [ ]:
if (!require(gtheory)) {
  install.packages("gtheory")
}

Loading required package: gtheory

Loading required package: lme4

Loading required package: Matrix


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

...

The following object is masked from 'package:ggplot2':

    last_plot

The following object is masked from 'package:stats':

    filter

The following object is masked from 'package:graphics':

    layout

  theory rater item score
1      1     1    1     5
2      2     1    1     9
3      3     1    1     3
4      4     1    1     7
5      5     1    1     9
6      6     1    1     3

    theory rater item score
115      5     3    4     3
116      6     3    4     2
117      7     3    4     4
118      8     3    4     1
119      9     3    4     1
120     10     3    4     1

## 2. G Study (Generalisability Study)

The G study estimates the amount of error caused by each facet (source
of error) and interaction of facets. The estimation represents the
variance expected if we were to sample one rater/item from the universe
admissible/available raters/items. This provides building blocks for the
D study.

In [ ]:
# One-facet design:
# Facets: raters
# g_study_formula <- score ~ (1 | theory) + (1 | rater)

# Two-facet design: includes all main effects and two-way interactions.
# Facets: raters and items
# Object: theories
g_study_formula <- score ~ (1 | theory) + (1 | rater) +
  (1 | item) + (1 | theory:rater) +
  (1 | theory:item) + (1 | rater:item)

g_study_results <- gstudy(data, g_study_formula)

boundary (singular) fit: see help('isSingular')

        source       var percent n
1  theory:item 0.4415520    10.0 1
2 theory:rater 0.0000000     0.0 1
3   rater:item 0.4538083    10.3 1
4       theory 0.5454735    12.3 1
5         item 0.3299985     7.5 1
6        rater 0.2202742     5.0 1
7     Residual 2.4259318    54.9 1

## 3. D Study (Decision Study)

The D study (re)estimates the variance components with specified numbers
of facets. Ideally, nearly all of the measured variance will be
attributed to the object of measurement (i.e., differences across
theories), with only a negligible amount of variance attributed to the
remaining facets (e.g., raters, items, and various interactions).

In [ ]:
# Run the D-study with the original numbers of raters and items
d_study_results <- dstudy(g_study_results, colname.objects = "theory",
                          colname.scores = "score", data = data)
d <- d_study_results$components
print(d)

        source        var percent  n
1  theory:item 0.11038799    10.5  4
2 theory:rater 0.00000000     0.0  3
3   rater:item 0.03781736     3.6 12
4       theory 0.54547350    51.9  1
5         item 0.08249961     7.8  4
6        rater 0.07342474     7.0  3
7     Residual 0.20216098    19.2 12

Plot the percentages of variance by source:

In [ ]:
variance_df <- data.frame(Facet = d[, "source"], Percent = d[, "percent"])
ggplotly(ggplot(variance_df, aes(x = Facet, y = Percent)) +
  geom_bar(stat = "identity", fill = "darkturquoise") +
  labs(x = "Source of Variance",
       y = "Percent") +
  theme_minimal(),
  tooltip = c("Percent"))

<a href="#fig-dstudy" class="quarto-xref">Figure 1</a> shows that
differences between theories are indeed the biggest source of variation
(51.9%), which indicates a potenially good reliability.

Interactions (theory:rater, etc.) show how the effects of one facet
depend on the levels of another. For example, theory:rater indicates how
much the rater variance/disagreement differs across different theories.

The next step is to quantify the reliability by calculating the
generalisability coefficient ($\varepsilon$) and dependability
coefficient ($\phi$) based on the estimated variance.

## 4. Generalisability Coefficient and Dependability Coefficient

The generalisability coefficient assesses the relative consistency or
reliability of the relative standing of individuals. For example, it
answers the question, “how reliably can we *rank* theories based on this
measurement design?” A higher coefficient (closer to 1) indicates higher
generalisability. A coefficient of 0 means none of the observed variance
is due to true score differences (it’s all error). A coefficient of 1
means the observed scores perfectly reflect the universe scores (no
error in ranking individuals).

The dependability coefficient is similar to the generalisability
coefficient, but it focuses on the absolute level of scores, rather than
just their rank - i.e., a theory’s score in relation to some absolute
standard or cut-off value, not just their position relative to others.
It answers the question, “How confident can we be in the *absolute
value* of this score?” A higher coefficient (closer to 1) indicates
greater dependability.

In [ ]:
cat("Generalisability coefficient =", d$generalizability, "\n")

Generalisability coefficient = 

Dependability coefficient =

A generalisability coefficient of 0.63 is resonably good, but could
potentially be improved further if needed.

The final step, depending on the purpose, is to predict and identify the
optimal numbers of facets to maximise the generalisability through D
studies.

## 5. Optimisation (Optional)

This optional D study explores different “designs” by changing the
number of levels for each facet (e.g., how many raters, how many items).
A grid search can be used to search for the optimal combinations of
raters and items to use for achieving a desired level of reliability.

In [ ]:
# Define a range of possible numbers of raters and items as alternatives
n_raters_range <- 1:10
n_items_range <- 1:10

# Generate combinations of n_raters_range and n_items_range
n_grid <- expand.grid(n_raters = n_raters_range,
                      n_items = n_items_range)

# Define function: Calculate generalisability coefficient
calculate_epsilon <- function(g_study_results, n_raters, n_items) {
  g <- g_study_results
  var_t <- g$var[g$source == "theory"]
  var_r <- g$var[g$source == "rater"]
  var_i <- g$var[g$source == "item"]
  var_tr <- g$var[g$source == "theory:rater"]
  var_ti <- g$var[g$source == "theory:item"]
  var_ri <- g$var[g$source == "rater:item"]
  var_resid <- g$var[g$source == "Residual"]
  if ((n_items | n_raters) == 0) {
    return(0)
  } else {
    # Relative error variance
    var_rel_error <- (var_tr / n_raters) + (var_ti / n_items) +
      (var_resid / (n_raters * n_items))
    epsilon <- var_t / (var_t + var_rel_error)
  }
}

# Define function: Calculate dependability coefficient
calculate_phi <- function(g_study_results, n_raters, n_items) {
  g <- g_study_results
  var_t <- g$var[g$source == "theory"]
  var_r <- g$var[g$source == "rater"]
  var_i <- g$var[g$source == "item"]
  var_tr <- g$var[g$source == "theory:rater"]
  var_ti <- g$var[g$source == "theory:item"]
  var_ri <- g$var[g$source == "rater:item"]
  var_resid <- g$var[g$source == "Residual"]
  if ((n_items | n_raters) == 0) {
    return(0)
  } else {
    # Absolute error variance
    var_abs_error <- (var_r / n_raters) + (var_i / n_items) +
      (var_tr / n_raters) + (var_ti / n_items) +
      (var_ri / (n_raters * n_items)) +
      (var_resid / (n_raters * n_items))
    phi <- var_t / (var_t + var_abs_error)
  }
}

# Calculate epsilon and phi for each combination
results <- n_grid %>%
  rowwise() %>%
  mutate(epsilon = calculate_epsilon(g, n_raters, n_items),
         phi = calculate_phi(g, n_raters, n_items))

print(results)

# A tibble: 100 × 4
# Rowwise: 
   n_raters n_items epsilon   phi
      <int>   <int>   <dbl> <dbl>
 1        1       1   0.160 0.123
 2        2       1   0.248 0.190
 3        3       1   0.304 0.232
 4        4       1   0.342 0.261
 5        5       1   0.371 0.282
 6        6       1   0.392 0.297
 7        7       1   0.409 0.310
 8        8       1   0.423 0.320
 9        9       1   0.434 0.328
10       10       1   0.444 0.335
# ℹ 90 more rows

Plot epsilon variation on a heatmap:

In [ ]:
plot_epsilon <- ggplot(results, 
                       aes(x = n_raters, y = n_items,
                                    fill = epsilon)) + 
  geom_tile() +
  scale_fill_viridis_c() +
  labs(x = "Number of Raters",
       y = "Number of Items",
       fill = "epsilon") +
  theme_bw()

plotly_epsilon <- ggplotly(plot_epsilon, 
                           tooltip = c("n_raters", "n_items", "epsilon"))

plotly_epsilon

And finally, plot phi variation on a heatmap:

In [ ]:
plot_phi <- ggplot(results, 
                   aes(x = n_raters, y = n_items, fill = phi)) +
  geom_tile() +
  scale_fill_viridis_c(option = "magma") +
  labs(x = "Number of Raters",
       y = "Number of Items",
       fill = "phi") +
  theme_bw()

plotly_phi <- ggplotly(plot_phi, 
                       tooltip = c("n_raters", "n_items", "phi"))

plotly_phi

<a href="#fig-plotly_epsilon" class="quarto-xref">Figure 2</a> and
<a href="#fig-plotly_phi" class="quarto-xref">Figure 3</a> show that it
could potentially achieve a generalisability coefficient \> 0.8 by
increasing raters and/or items, e.g., with combinations such as \[3
raters, 10 items\], \[5 raters, 7 items\] or \[10 raters, 6 items\],
assuming variance components remain unchanged.

### References

Brennan, R. L. (2001). *Generalizability Theory*. Springer New York.
<https://doi.org/10.1007/978-1-4757-3456-0>

Huebner, A., & Lucht, M. (2019). Generalizability theory in r.
*University of Massachusetts Amherst*.
<https://doi.org/10.7275/5065-GC10>